In [11]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn import neighbors, metrics
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
red_wine = pd.read_csv('RedWineSampleData.csv')
red_wine.dropna()

rename_columns = {
    'fixed acidity': 'fixed_acidity',
    'volatile acidity': 'volatile_acidity',
    'citric acid': 'citric_acid',
    'residual sugar': 'residual_sugar',
    'chlorides': 'chlorides',
    'free sulfur dioxide': 'free_sulfur_dioxide',
    'total sulfur dioxide': 'total_sulfur_dioxide',
    'density': 'density',
    'pH': 'pH',
    'sulphates': 'sulphates',
    'alcohol': 'alcohol'
}

red_wine = red_wine.rename(columns=rename_columns)

print(len(red_wine))

2752


In [3]:
red_wine['target'] = 0.0

red_wine['target'] = (red_wine['quality'] >= 1).astype(int)
red_wine['target'].value_counts()

0    1376
1    1376
Name: target, dtype: int64

In [4]:
Y, X = dmatrices('target ~ 0 + fixed_acidity + volatile_acidity + citric_acid'
       '+ residual_sugar + chlorides + free_sulfur_dioxide'
       '+ total_sulfur_dioxide + density + pH + sulphates+ alcohol',
                 red_wine,return_type="dataframe")

print('Design matrix X')
print(X[:5])
print('Outcomes Y')
print(Y[:5])

Design matrix X
   fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  
0      9.4  
1      9.8  
2      9.8  
3      9.8  
4    

In [5]:
y = Y['target'].values
print(y[5:])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

y_train[:5]
red_model = neighbors.KNeighborsClassifier(n_neighbors=10,
                                       weights='uniform',
                                       p=2)

red_model.fit(X_train, y_train)
X_train.iloc[0:5]
print('Predictions =', red_model.predict(X_train.iloc[0:5]))
y_train[0:5]
red_model.predict_proba(X_train.iloc[0:5])


[0. 0. 1. ... 1. 1. 1.]
Predictions = [0. 0. 1. 1. 1.]


array([[0.7, 0.3],
       [1. , 0. ],
       [0.2, 0.8],
       [0. , 1. ],
       [0. , 1. ]])

In [6]:
prediction_on_training = red_model.predict(X_train)
print("Training accuracy score: ", metrics.accuracy_score(y_train, prediction_on_training))

predicted_classes = red_model.predict(X_test)
print("Test accuracy score: ", metrics.accuracy_score(y_test, predicted_classes))

Training accuracy score:  0.877466251298027
Test accuracy score:  0.8353510895883777


In [7]:
red_model.predict_proba(X_train.iloc[0:5])
#get scores

red_model.predict(X_train.iloc[0:5])
#get classes

array([0., 0., 1., 1., 1.])

In [8]:
from sklearn import model_selection

def cv_accuracy_score(X, y, nn):
    red_model = neighbors.KNeighborsClassifier(n_neighbors=nn,
                                           weights='uniform')
    accuracies = []
    
    kfold = model_selection.StratifiedKFold(n_splits=3, shuffle=True).split(X, y)
    for train, holdout in kfold:
        X_train = X.iloc[train]
        X_holdout = X.iloc[holdout]
        y_train = y[train]
        y_holdout = y[holdout]
        red_model.fit(X_train, y_train)
        accuracy = metrics.accuracy_score(y_holdout, red_model.predict(X_holdout))
        accuracies.append(accuracy)
    
    average_accuracy = Series(accuracies).mean()
    return average_accuracy

cv_accuracy_score(X, y, nn=1)
nn = [1] + [(x + 1) * 5 for x in range(19)]
print(nn)

[1, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95]


In [9]:
scores = [cv_accuracy_score(X, y, num_nbrs) for num_nbrs in nn]

In [10]:
plot(nn, scores)
xlabel('Number of nearest neighbors')
ylabel('Accuracy of classifier')

NameError: name 'plot' is not defined